In [1]:
# Basic semantic search example.
# Semantics depends upon word context. Thus,
# to perform semantic search a simple embedding
# representation can be used for words.
#
# Author: Fabrício Galende Marques de Carvalho

from pandas import DataFrame


documents= [
    "entrega rápida para são josé dos campos",
    "valor da entrega para meu bairro",
    "comida muito saborosa",
    "prato delicioso e muito saboroso",
    "o custo de vida está muito elevado",
    "custa caro morar em são josé dos campos"
]

# E algumas classes correspondentes
labels = ["entrega", "entrega", "comida", "comida", "custo", "custo"]

# Criar o DataFrame
sentence_dataframe = DataFrame({  "docs": documents,
    "category": labels})
print(sentence_dataframe)


                                      docs category
0  entrega rápida para são josé dos campos  entrega
1         valor da entrega para meu bairro  entrega
2                    comida muito saborosa   comida
3         prato delicioso e muito saboroso   comida
4       o custo de vida está muito elevado    custo
5  custa caro morar em são josé dos campos    custo


In [2]:
import re
import numpy as np
from numpy import dot 
from numpy.linalg import norm
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

text_corpus = "Uma entrega rápida sempre deixa o cliente satisfeito " \
              "e sempre é possível em são josé dos Campos. "\
              "Custo depende do preço das coisas. Custo de vida alto " \
              "sempre quer dizer coisas caras. Aquilo que comemos pode ser " \
              "uma comida muito gostosa. Comida que comemos nos faz bem. "\
              "Comida e alimento saudável sempre nos fazem bem. Alimento e " \
              "comida boa é o que todos querem. "
              
print(text_corpus)
tokenizer = sent_tokenize
# 1. Basic sentence tokens building
sentence_tokens = tokenizer(text_corpus)
print("Sentence tokens: ")
print(sentence_tokens)
print("\n\n")
# 2. Basic word tokens building
word_tokens = [ word_tokenize(sentence)  for sentence in sentence_tokens]
print("Word tokens: ")
print(word_tokens)
print("\n\n")

# 3. Removing punctuation characters and converting
# all charecters to lower case:
normalized_sentences = []
i = 0
for sentence in word_tokens:
    normalized_sentences.append([])
    for word in sentence:
        word = re.sub(r'[^A-Za-zÀ-Ýà-ý]','', word).lower()
        if word!='':
            normalized_sentences[i].append(word)
    i = i+1

print("\n\n")
print("Lower case, punctuation free word tokens")
print(normalized_sentences)

# 4. Stop word removal:
stop_words = ['a', 'as', 'e', 'o', 'os', 'da', 'de', 'do', 'um', 'uma']
for word in stop_words:
    for sentence in normalized_sentences:
        if word in sentence:
            print(word)
            print(sentence)
            sentence.remove(word)
print("\n\n")
print("Normalized text: ")
print(normalized_sentences)

# 4. building the word2vec model
# Model configuration
feature_size = 32  # size of vector representation
window_context = 3
min_word_count = 1
sample = 1e-3
w2vec_repr = word2vec.Word2Vec(normalized_sentences, vector_size= feature_size,
                                window=window_context, min_count= min_word_count,
                                sample=sample, epochs = 50)

Uma entrega rápida sempre deixa o cliente satisfeito e sempre é possível em são josé dos Campos. Custo depende do preço das coisas. Custo de vida alto sempre quer dizer coisas caras. Aquilo que comemos pode ser uma comida muito gostosa. Comida que comemos nos faz bem. Comida e alimento saudável sempre nos fazem bem. Alimento e comida boa é o que todos querem. 
Sentence tokens: 
['Uma entrega rápida sempre deixa o cliente satisfeito e sempre é possível em são josé dos Campos.', 'Custo depende do preço das coisas.', 'Custo de vida alto sempre quer dizer coisas caras.', 'Aquilo que comemos pode ser uma comida muito gostosa.', 'Comida que comemos nos faz bem.', 'Comida e alimento saudável sempre nos fazem bem.', 'Alimento e comida boa é o que todos querem.']



Word tokens: 
[['Uma', 'entrega', 'rápida', 'sempre', 'deixa', 'o', 'cliente', 'satisfeito', 'e', 'sempre', 'é', 'possível', 'em', 'são', 'josé', 'dos', 'Campos', '.'], ['Custo', 'depende', 'do', 'preço', 'das', 'coisas', '.'], ['Cu

In [3]:
# Let's check the semantic search:
v1 = w2vec_repr.wv["alimento"]
v2 = w2vec_repr.wv["entrega"]
v3 = w2vec_repr.wv["comida"]
v4 = w2vec_repr.wv["custo"]

print("Cosine similarity between 'alimento' and 'entrega': ")
print(dot(v1,v2)/(norm(v1)*norm(v2)))

print("Cosine similarity between 'alimento' and 'comida': ")
print(dot(v1,v3)/(norm(v1)*norm(v3)))

print("Cosine similarity between 'alimento' and 'custo': ")
print(dot(v1,v4)/(norm(v1)*norm(v4)))

# If a database search is to be performed (suppose that pandas dataframe behaves like a db search)
print("\nSearch result (i.e., API response o database query): ")
print("Result: ")
print(sentence_dataframe[ sentence_dataframe["category"] == "comida"])

Cosine similarity between 'alimento' and 'entrega': 
0.049173858
Cosine similarity between 'alimento' and 'comida': 
0.076921314
Cosine similarity between 'alimento' and 'custo': 
-0.005212678

Search result (i.e., API response o database query): 
Result: 
                               docs category
2             comida muito saborosa   comida
3  prato delicioso e muito saboroso   comida
